# T4 - Final Project

Semester 2221, CSEC 520/620, Team 4\
Final Project - URL Classification\
Due by December 14, 2022 11:59 PM EST.\
Accounts for 18% of total grade.

In [ ]:
# Ensure the WHOIS package is installed
print(f'{"":#^{36}}\n{"## Installing Packages ":#<{36}}\n{"":#^{36}}')
!pip install python-whois

print(f'\n{"":#^{36}}\n{"## Updating Repository ":#<{36}}\n{"":#^{36}}')
# Download our repo, which contains the RAW datasets
!git init
!git remote add origin https://github.com/aisgbnok/T4-Project.git
!git pull origin main --allow-unrelated-histories

In [ ]:
import os
import pandas as pd
import whois
from functools import reduce
from timeit import default_timer as timer

In [ ]:
start = timer()
def load_data():

  # Get all of our datasets
  data_frames = []

  for rt, dirs, files in os.walk("datasets"):
    files.remove("README.md")
    for f_name in files:
      path = os.path.join(rt, f_name)
      dataframe = pd.read_csv(path)
      data_frames.append(dataframe)

  # Rename our columns
  for i in range(0, len(data_frames)):
    display(data_frames[i])
    data_frames[i].columns = data_frames[0].columns
    display(data_frames[i])

  return reduce(lambda x, y: pd.merge(x, y), data_frames)

data = load_data()
end = timer()
print(end - start)

display(data)

In [ ]:
phish_files = pd.read_csv('malicious_phish.csv')
display(phish_files)

In [ ]:
mal_v_non = pd.read_csv('malicious_v_nonmalicious.csv')
display(mal_v_non)

In [ ]:
# Merging the datasets, and removing the type column (for now)
merged = pd.merge(phish_files, mal_v_non, how='outer', on='url')
merged.loc[merged["type"] == "benign", "label"] = "good"
merged.loc[merged["type"] != "benign", "label"] = "bad"
merged = merged.drop(columns=["type"])
display(merged)

In [ ]:
#multithreading the data set
def multithreading():
  perc = 0.0
  threads = []
  max_threads = 10
  thread_num = 1
  while perc < 1:
    if len(threads) >= max_threads:
      for thread in threads:
        thread.join()
        print("Thread "+str(thread_num)+" done")
        thread_num+=1
    print("perc "+str(perc))
    thread = threading.Thread(target=checker, args=(perc*100, merged.iloc[int(perc*len(merged)):int((perc+0.01)*len(merged))]))
    thread.start()
    threads.append(thread)
    perc = perc + 0.01

In [ ]:
#numsection -> the percent into the data that it is
#data -> the data of the current section
def checker(numsection, data):
  rows = []
  for index, row in data.iterrows():
    w = None
    try:
      w = whois.whois(data.iloc[index]["url"])
    except:
      continue
    if w.domain_name != "null" and w.domain_name is not None:
      rowdict = {}
      rowdict["originalurl"] = data.iloc[index]["url"]
      rowdict.update(dict(w))
      rows.append(rowdict)
  whoisframe = pd.DataFrame(rows)
  whoisframe.to_csv('csv/whoisdata' + str(numsection) + '.csv')
  print("csv made")

In [ ]:
#multithreading()

In [ ]:
os.chdir('/content/drive/Shareddrives/CSEC 620 Group 4/Final Project/csv')
filelist = os.listdir('.')
newframe = pd.DataFrame()
for f in filelist:
  if f.find('whoisdata') == 0:
    currentframe = pd.read_csv(f)
    newframe = pd.concat([newframe, currentframe], ignore_index=True)

In [ ]:
nummal = 0
numbenign = 0
for index, row in newframe.iterrows():
  thisurl = row["originalurl"]
  mergedrow = merged.loc[merged["url"] == thisurl]
  benigncheck = mergedrow.iloc[0]["label"]
  if benigncheck == "good":
    numbenign = numbenign + 1
  else:
    nummal = nummal + 1
print(nummal)
print(numbenign)

# Splitting the data set

In [ ]:
splitVal = int(len(newframe)*.2)
testing_set = newframe.iloc[:splitVal,]
training_set = newframe.iloc[splitVal:,]

In [ ]:
countries = {}
for index, row in training_set.iterrows():
  rowcountry = row["country"]
  if rowcountry in countries.keys():
    countries[rowcountry] = countries[rowcountry] + 1
  else:
    countries[rowcountry] = 1

for c in countries:
  print(c, countries[c])

In [ ]:
regs = {}
for index, row in training_set.iterrows():
  rowreg = row["registrar"]
  if rowreg in regs.keys():
    regs[rowreg] = regs[rowreg] + 1
  else:
    regs[rowreg] = 1

for r in regs:
  print(r, regs[r])

In [ ]:
dns = {}
for index, row in training_set.iterrows():
  rowdns = row["dnssec"]
  if rowdns in dns.keys():
    dns[rowdns] = dns[rowdns] + 1
  elif 'unsigned' in str(rowdns).lower() and (rowdns != 'unsigned'):
    dns["unsigned"] = dns["unsigned"] + 1
  else:
    dns[rowdns] = 1

for r in dns:
  print(r, dns[r])

In [ ]:
from numpy.lib.function_base import percentile
percentages = {}
percentagerow = []

for index, row in training_set.iterrows():
  rowdomain = row["domain_name"]
  num = 0
  lett = 0
  for char in rowdomain:
    if char.isalpha():
      lett += 1
    elif char.isnumeric():
      num += 1
  perc = round(num/(num+lett), 1)
  percentagerow.append(perc)
  if perc in percentages.keys():
    percentages[perc] = percentages[perc] + 1
  else:
    percentages[perc] =  1

for p in percentages:
  print(p, percentages[p])

In [ ]:
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
le1 = preprocessing.LabelEncoder()
url1 = le1.fit_transform(training_set["originalurl"])
countrylabel = le1.fit_transform(training_set["country"])
model1 = GaussianNB()
model1.fit(url1.reshape(-1, 1), countrylabel)

In [ ]:
le2 = preprocessing.LabelEncoder()
url2 = le2.fit_transform(training_set["originalurl"])
reglabel = le2.fit_transform(training_set["registrar"])
model2 = GaussianNB()
model2.fit(url2.reshape(-1, 1), reglabel)

In [ ]:
le3 = preprocessing.LabelEncoder()
url3 = le3.fit_transform(training_set["originalurl"])
dnslabel = le3.fit_transform(training_set["dnssec"])
model3 = GaussianNB()
model3.fit(url3.reshape(-1, 1), dnslabel)

In [ ]:
le4 = preprocessing.LabelEncoder()
url4 = le4.fit_transform(training_set["originalurl"])
percentlabel = le4.fit_transform(percentagerow)
model4 = GaussianNB()
model4.fit(url4.reshape(-1, 1), percentlabel)